# <center>**Import necessary libraries**</center>


In [32]:
import pathlib
import pandas as pd
import math
import design_functions.rebar_information as rebar_func

## <center>Import and read the desired beam flexural and shear reinforcement excel file from ETABS</center>

In [33]:
excel_file = pathlib.Path('excel_files\example_analysis.xlsx')
initial_df = pd.read_excel(excel_file, sheet_name=None)

## <center>Because the excel sheet has two sheets, flexural and shear, they need to be read individually</center>

In [34]:
initial_df['Sheet1'].head()

,TABLE: Concrete Beam Flexure Envelope - ACI 318-19,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,Story,Label,UniqueName,Section,Location,(-) Moment,(-) Combo,As Top,(+) Moment,(+) Combo,As Bot
1,NaN,NaN,NaN,NaN,NaN,kN-m,NaN,mm²,kN-m,NaN,mm²
2,P4,B213,1366,B300X500-C45/56,End-I,0,U6DLE-3,3,3.5997,U6DLEH-4,28
3,P4,B213,1366,B300X500-C45/56,Middle,-28.3344,U6DLE-4,229,0,U6DLE-4,90
4,P4,B213,1366,B300X500-C45/56,End-J,-41.1314,U1DT-ve,302,0,U6DLE-4,124


In [35]:
initial_df['Sheet2'].head()

,TABLE: Concrete Beam Shear Envelope - ACI 318-19,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13
0,Story,Label,UniqueName,Section,Location,V,V Combo,At,T for At,T for At Combo,At Torsion,T for As,T for As Combo,As Torsion
1,NaN,NaN,NaN,NaN,NaN,kN,NaN,mm²/m,kN-m,NaN,mm²/m,kN-m,NaN,mm²
2,P4,B213,1366,B300X500-C45/56,End-I,20.189,U5DW-,0,0.0008,U5DW-,0,0.0008,U5DW-,0
3,P4,B213,1366,B300X500-C45/56,Middle,25.4169,U5DW-,0,0.2493,U5DW-,0,0.2493,U5DW-,0
4,P4,B213,1366,B300X500-C45/56,End-J,30.0491,U5DW-,0,0.2493,U5DW-,0,0.2493,U5DW-,0


## <center>Create table to store beam reinforcement schedule</center>

In [36]:
# Create dictionary which reflects column headings
columns = pd.MultiIndex.from_tuples([
    ('Dimensions', 'Width (mm)'),
    ('Dimensions', 'Depth (mm)'),
    ('Bottom Reinforcement', 'Left (BL)'),
    ('Bottom Reinforcement', 'Middle (B)'),
    ('Bottom Reinforcement', 'Right (BR)'),
    ('Top Reinforcement', 'Left (TL)'),
    ('Top Reinforcement', 'Middle (T)'),
    ('Top Reinforcement', 'Right (TR)'),
    ('Shear links', 'Left (H)'),
    ('Shear links', 'Middle (J)'),
    ('Shear links', 'Right (K)'),
    ('Side Face Reinforcement', '')
])
beam_schedule_df = pd.DataFrame(columns=columns)
beam_schedule_df

Empty DataFrame
Columns: [(Dimensions, Width (mm)), (Dimensions, Depth (mm)), (Bottom Reinforcement, Left (BL)), (Bottom Reinforcement, Middle (B)), (Bottom Reinforcement, Right (BR)), (Top Reinforcement, Left (TL)), (Top Reinforcement, Middle (T)), (Top Reinforcement, Right (TR)), (Shear links, Left (H)), (Shear links, Middle (J)), (Shear links, Right (K)), (Side Face Reinforcement, )]
Index: []

In [37]:
# new_row = {
#     ('Dimensions', 'Width (mm)'): 200,
#     ('Dimensions', 'Depth (mm)'): 300,
#     ('Bottom Reinforcement', 'Left (BL)'): 4,
#     ('Bottom Reinforcement', 'Middle (B)'): 3,
#     ('Bottom Reinforcement', 'Right (BR)'): 5,
#     ('Top Reinforcement', 'Left (TL)'): 1,
#     ('Top Reinforcement', 'Middle (T)'):1,
#     ('Top Reinforcement', 'Right (TR)'):1,
#     ('Shear links', 'Left (H)'):1,
#     ('Shear links', 'Middle (J)'):1,
#     ('Shear links', 'Right (K)'):1,
#     ('Side Face Reinforcement', ''):1
# }
# beam_schedule_df.loc[0] = new_row
# beam_schedule_df